In [8]:
import pandas as pd
import sys
import os
import json
from tqdm import tqdm
sys.path.append('/Users/yijingyang/Library/CloudStorage/OneDrive-个人/GradPilot/ProgramDB/DataScience')
import asyncio
from call_api import call_gemini, async_call_gemini
from tqdm.asyncio import tqdm_asyncio

field_name = "托福送分ETS code"
field_path = f"/Users/yijingyang/Library/CloudStorage/OneDrive-个人/GradPilot/ProgramDB/DataScience/fields_csv/{field_name}.csv"
field_df = pd.read_csv(field_path)

field_df.shape

(351, 7)

In [10]:
prompt_template = """
You are an assistant that checks TOFEL ETS code for the following graduate program, which is used for applicants to send TOFEL scores to the university.

1. Search the admissions and program webpages provided.  
2. Use Google search to find more information.

Answer instructions:
If you find the ETS code, just return the ETS code and nothing else. If not found, return "Not found".

Example response:
"Not found"
"2162"
"7746"
"2235"

Use Google to search **"{university} {degree} {program} {department} TOFEL ETS code"** for more information.

URLs you should check:
• Admissions URL: {admissions_url}  
• Program URL: {program_url}

Here are your response. Note that either return the ETS code or "Not found", with nothing else:
"""

In [12]:
semaphore = asyncio.Semaphore(2) 

async def process_row(row, prompt_template, num_vote, model_name):
    async with semaphore:
        row = row.to_dict()
        prompt = prompt_template.format(
            university     = row["大学英文名称"],
            degree         = row["学位"],
            program        = row["专业英文名称"],
            department     = row["所属院系"],
            admissions_url = row["招生网址"],
            program_url    = row["专业网址"],
        )
        record = row.copy()
        record["llm_reponses"] = {}

        # Launch all API calls in parallel for this row
        tasks = [
            async_call_gemini(prompt, model_name=model_name, use_search=True, url_context=True)
            for _ in range(num_vote)
        ]
        responses = await asyncio.gather(*tasks)

        for i, response in enumerate(responses):
            try:
                text = response.candidates[0].content.parts[0].text
            except:
                text = ''
            try:
                url_context = str(response.candidates[0].url_context_metadata)
            except:
                url_context = "Not used"
            try:
                search_pages = f"Search Chunks: {response.candidates[0].grounding_metadata.grounding_chunks}"
            except:
                search_pages = "Not used"
            try:
                search_queries = f"Search Query: {response.candidates[0].grounding_metadata.web_search_queries}"
            except:
                search_queries = "Not used"
            try:
                search_support = f"Search Query: {response.candidates[0].grounding_metadata.groundingSupports}"
            except:
                search_support = "Not used"

            record["llm_reponses"][f"response {i+1}"] = {
                "response_text": text,
                "url_context": url_context,
                "search_queries": search_queries,
                "search_pages": search_pages,
                "search_support": search_support,
            }
        return record

async def request_and_store_async(prompt_template, field_df, num_vote, model_name, start_from=0, end_at=-1):
    df = field_df.copy()[start_from:end_at]
    response_records = []

    # Create tasks for all rows
    tasks = [
        process_row(row, prompt_template, num_vote, model_name)
        for _, row in df.iterrows()
    ]
    # Run all row tasks in parallel (limit concurrency if needed)
    response_records = await tqdm_asyncio.gather(*tasks)

    # Save results
    with open(f"../fields_records/{field_name}/{field_name}_{model_name}_{start_from}_{end_at}.json", "w") as f:
        json.dump(response_records, f, ensure_ascii=False, indent=2)
    return response_records

In [14]:
import nest_asyncio
nest_asyncio.apply()  # Only needed in Jupyter

num_vote = 3
start_from = 0
end_at = -1
model_name = "gemini-2.5-flash"
response_records = asyncio.run(
    request_and_store_async(prompt_template, field_df, num_vote, model_name, start_from=start_from, end_at=end_at)
)

100%|██████████| 350/350 [43:38<00:00,  7.48s/it]


In [1]:
import json
import pandas as pd
import re
import os

def extract_toefl_ets_code_info(json_file_path):
    """
    从JSON文件中提取托福送分ETS code信息
    
    Args:
        json_file_path: JSON文件路径
    
    Returns:
        DataFrame: 处理后的数据
    """
    
    # 读取JSON文件
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    results = []
    
    for record in data:
        # 提取基本信息
        basic_info = {
            '大学英文名称': record.get('大学英文名称', '').strip(),
            '学位': record.get('学位', ''),
            '专业英文名称': record.get('专业英文名称', ''),
            '所属院系': record.get('所属院系', ''),
            '招生网址': record.get('招生网址', ''),
            '专业网址': record.get('专业网址', ''),
        }
        
        # 获取三个LLM responses
        llm_responses = record.get('llm_reponses', {})
        response_1 = llm_responses.get('response 1', {}).get('response_text', '').strip()
        response_2 = llm_responses.get('response 2', {}).get('response_text', '').strip()
        response_3 = llm_responses.get('response 3', {}).get('response_text', '').strip()
        
        def extract_ets_code(response_text):
            """
            从回答中提取ETS code
            返回: ETS code字符串 或 'invalid'
            """
            if not response_text or len(response_text.strip()) == 0:
                return 'invalid'
            
            # 清理文本（移除换行符和多余空格）
            cleaned_text = re.sub(r'\s+', ' ', response_text).strip()
            
            # 检查是否为无效回答（包含解释性文字、过程描述等）
            invalid_patterns = [
                r'not found',
                r'not mentioned',
                r'not available',
                r'not specified',
                r'unable to find',
                r'cannot find',
                r'no information',
                r'please check',
                r'contact the',
                r'visit the',
                r'according to',
                r'based on',
                r'the ets code',
                r'toefl.*code.*is',
                r'code.*for.*toefl'
            ]
            
            # 如果包含解释性文字，认为是无效回答
            for pattern in invalid_patterns:
                if re.search(pattern, cleaned_text, re.IGNORECASE):
                    return 'invalid'
            
            # 如果回答过长（超过10个字符），可能包含解释，视为无效
            if len(cleaned_text) > 10:
                return 'invalid'
            
            # 提取纯数字ETS code（通常是4位数字）
            ets_pattern = r'^\d{4}$'
            if re.match(ets_pattern, cleaned_text):
                return cleaned_text
            
            # 如果文本中只包含数字和少量符号，尝试提取数字
            number_only = re.sub(r'[^\d]', '', cleaned_text)
            if len(number_only) == 4 and number_only.isdigit():
                return number_only
            
            # 其他情况视为无效
            return 'invalid'
        
        # 分类三个回答
        code_1 = extract_ets_code(response_1)
        code_2 = extract_ets_code(response_2)
        code_3 = extract_ets_code(response_3)
        
        # 收集有效的ETS codes
        valid_codes = [code for code in [code_1, code_2, code_3] if code != 'invalid']
        
        # 判断最终结果
        if len(valid_codes) == 0:
            # 所有回答都无效
            final_decision = "需要额外确认"
            decision_status = "所有回答都无效"
        elif len(valid_codes) == 1:
            # 只有一个有效回答
            final_decision = "需要额外确认"
            decision_status = "只有一个有效回答，不足以确认"
        elif len(valid_codes) == 2:
            # 有两个有效回答
            if valid_codes[0] == valid_codes[1]:
                # 两个相同
                final_decision = valid_codes[0]
                decision_status = "两个有效回答一致"
            else:
                # 两个不同
                final_decision = "需要额外确认"
                decision_status = f"两个有效回答不一致: {valid_codes[0]} vs {valid_codes[1]}"
        else:
            # 三个都有效
            if code_1 == code_2 == code_3:
                # 三个完全一致
                final_decision = code_1
                decision_status = "三个回答完全一致"
            elif len(set(valid_codes)) == 1:
                # 所有有效回答一致（理论上和上面情况相同，但防御性编程）
                final_decision = valid_codes[0]
                decision_status = "所有有效回答一致"
            else:
                # 有不同的回答
                final_decision = "需要额外确认"
                code_counts = {}
                for code in valid_codes:
                    code_counts[code] = code_counts.get(code, 0) + 1
                decision_status = f"回答不一致: {code_counts}"
        
        # 将所有信息合并
        result = basic_info.copy()
        result.update({
            '托福送分ETS code': final_decision,
            '判断状态': decision_status,
            '有效回答数': len(valid_codes),
            '总回答数': len([r for r in [response_1, response_2, response_3] if r.strip()]),
            'Response 1 提取': code_1,
            'Response 2 提取': code_2,
            'Response 3 提取': code_3,
            'Response 1 原文': response_1[:50] + ('...' if len(response_1) > 50 else ''),
            'Response 2 原文': response_2[:50] + ('...' if len(response_2) > 50 else ''),
            'Response 3 原文': response_3[:50] + ('...' if len(response_3) > 50 else ''),
        })
        
        results.append(result)
    
    # 转换为DataFrame
    df = pd.DataFrame(results)
    
    # 保存CSV文件到同一目录
    output_dir = os.path.dirname(json_file_path)
    csv_filename = os.path.basename(json_file_path).replace('.json', '_processed.csv')
    output_path = os.path.join(output_dir, csv_filename)
    
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"处理完成！结果已保存到: {output_path}")
    print(f"共处理 {len(df)} 条记录")
    print(f"\n托福送分ETS code分布:")
    print(df['托福送分ETS code'].value_counts(dropna=False))
    print(f"\n判断状态分布:")
    print(df['判断状态'].value_counts())
    
    # 显示一些统计信息
    confirmed_codes = df[df['托福送分ETS code'] != '需要额外确认']
    need_confirmation = df[df['托福送分ETS code'] == '需要额外确认']
    
    print(f"\n统计信息:")
    print(f"成功确认ETS code的项目: {len(confirmed_codes)} 个")
    print(f"需要额外确认的项目: {len(need_confirmation)} 个")
    print(f"确认率: {len(confirmed_codes)/len(df)*100:.1f}%")
    
    return df


json_file_path = "/Users/yijingyang/Library/CloudStorage/OneDrive-个人/GradPilot/ProgramDB/DataScience/fields_records/托福送分ETS code/托福送分ETS code_gemini-2.5-flash_0_-1.json"
df = extract_toefl_ets_code_info(json_file_path)

# 显示前几行数据预览
print("\n数据预览:")
print(df[['大学英文名称', '学位', '专业英文名称', '托福送分ETS code', '判断状态']].head(10))

# 显示需要额外确认的案例
need_confirmation_cases = df[df['托福送分ETS code'] == '需要额外确认']
if len(need_confirmation_cases) > 0:
    print(f"\n需要额外确认的案例示例（前5个）:")
    for idx, row in need_confirmation_cases.head(5).iterrows():
        print(f"{row['大学英文名称']} - {row['专业英文名称']}: {row['判断状态']}")

处理完成！结果已保存到: /Users/yijingyang/Library/CloudStorage/OneDrive-个人/GradPilot/ProgramDB/DataScience/fields_records/托福送分ETS code/托福送分ETS code_gemini-2.5-flash_0_-1_processed.csv
共处理 350 条记录

托福送分ETS code分布:
托福送分ETS code
需要额外确认    79
3665      12
4852      10
1836       9
1839       9
          ..
5859       1
3679       1
3915       1
2660       1
5969       1
Name: count, Length: 92, dtype: int64

判断状态分布:
判断状态
三个回答完全一致                         195
两个有效回答一致                          76
只有一个有效回答，不足以确认                    41
所有回答都无效                           31
回答不一致: {'3434': 1, '3451': 2}      1
回答不一致: {'2176': 2, '2074': 1}      1
两个有效回答不一致: 2176 vs 2074            1
回答不一致: {'3087': 2, '8675': 1}      1
回答不一致: {'4999': 2, '3667': 1}      1
回答不一致: {'2927': 2, '0404': 1}      1
回答不一致: {'2790': 2, '2765': 1}      1
Name: count, dtype: int64

统计信息:
成功确认ETS code的项目: 271 个
需要额外确认的项目: 79 个
确认率: 77.4%

数据预览:
                       大学英文名称     学位  \
0          Harvard University     MS   
1          Ha